## Check uniqueness and record relationships

Simple QC notebook to check the record counts, uniquenes, and relationship types between entities.

In [ ]:
"""Uncomment if running in Azure ML"""
# import sys
# sys.path.append(r'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jopm/code/Users/JOPM/osdu_jopm_rottation')
# sys.path

In [2]:

import os
import json
from libs.ladmark_service.landmark_client import LandmarkService
from libs.utilities import unique_ids_counts, get_non_unique_values, count_relationships, compare_dictionaries

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

#### Get records counts from SDB created before last runtime

In [3]:
last_run_date_time = "2023-07-26T09:10:16.945Z"
last_run_date_time = None
query_records_created_prior_latsrundate = True

osdu_env = "npequinor-dev"
source = "sdb"

In [4]:
entities = {
    "seismicsurveys": {
        # Attibutes to make the sourse API request
        "relativePath": "seismicsurveys",
        "filters_dict": {"surveyDimension": ["eq", "'3D'"]},
        "return_cols": ["surveyId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "surveyId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:seismicsurveys:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:master-data--SeismicAcquisitionSurvey:1.2.0"]
    },
    "seismicprojects": {
        # Attibutes to make the sourse API request
        "relativePath": "seismicprojects",
        "filters_dict": {"seismicDimension": ["eq", "'3D'"]},
        "return_cols": ["projectId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "projectId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:seismicprojects:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:master-data--SeismicProcessingProject:1.2.0"],
    },
    "navigationsets": {
        # Attibutes to make the sourse API request
        "relativePath": "navigationsets",
        "filters_dict": {"surveyDimension": ["eq", "'3D'"]},
        "return_cols": ["navigationSetId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "navigationSetId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:navigationsets:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:dataset--FileCollection.Generic:1.0.0"]
    },
    "binsetgrids": {
        # Attibutes to make the sourse API request
        "relativePath": "binsetgrids",
        "filters_dict": None,
        "return_cols": ["binsetGrid3dId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "binsetGrid3dId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:binsetgrids:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:work-product-component--SeismicBinGrid:1.0.0"]
    },
    "seismicpoststackdatasets": {
        # Attibutes to make the sourse API request
        "relativePath": "seismicpoststackdatasets",
        "filters_dict": {"surveyDimension": ["eq", "'3D'"]},
        "return_cols": ["seismicPoststackDatasetId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "seismicPoststackDatasetId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:seismicpoststackdatasets:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:dataset--FileCollection.SEGY:1.0.0", "osdu:wks:work-product-component--SeismicTraceData:1.3.0"]
    },
    "poststackcubes": {
        # Attibutes to make the sourse API request
        "relativePath": "poststackcubes",
        "filters_dict": None,
        "return_cols": ["poststackCubeId", "seismicPoststackDatasetId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "poststackCubeId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:poststackcubes:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:work-product-component--SeismicTraceData:1.3.0"]
    },
    "poststackcubegeometries": {
        # Attibutes to make the sourse API request
        "relativePath": "poststackcubegeometries",
        "filters_dict": None,
        "return_cols": ["poststackCubeId", "seismicPoststackDatasetId"],
        "expand_on": None,
        "lastrundatetime": last_run_date_time,
        "before_rundate": query_records_created_prior_latsrundate,
        "enitity_unique_identifier": "poststackCubeId",
        
        # Attributes to connect to osdu
        "custom_schema": f"eqnr:iEnergy-{source}:poststackcubegeometries:1.0.0",
        "osdu_well_known_schema": ["osdu:wks:work-product-component--SeismicTraceData:1.3.0", "osdu:wks:dataset--FileCollection.SEGY:1.0.0"]
    }
}

#### Get record counts from SDB

In [5]:
"""
Let's first get the records counts originally given by the SOURCE API.
"""

source_records_count = dict()

with LandmarkService(
    os.environ[f"{source}_metadata_url"],
    os.environ[f"{source}_ds_security_url"],
    os.environ[f"{source}_refresh_token"]
    ) as landmark:
    
    for entity_name in entities:
        records_count = landmark.get_count(**entities.get(entity_name))
        source_records_count[entity_name] = records_count
        
        print(f"{entity_name.upper()} records count: {source_records_count.get(entity_name)}")

    ## TODO: Leave the query by batch option commented out as it might be needed when we switch into requesting data from DISKOS (larger data)        

SEISMICSURVEYS records count: 1979
SEISMICPROJECTS records count: 2408
NAVIGATIONSETS records count: 3257
BINSETGRIDS records count: 2148
SEISMICPOSTSTACKDATASETS records count: 19908
POSTSTACKCUBES records count: 19829
POSTSTACKCUBEGEOMETRIES records count: 19829


#### Get actual records from SDB created before last runtime

In [6]:
"""
Requesting records from source system.
"""

source_records = dict()

with LandmarkService(os.environ[f"{source}_metadata_url"], os.environ[f"{source}_ds_security_url"], os.environ[f"{source}_refresh_token"]) as landmark:
    
    for entity_name in entities:
        records = landmark.get_records(**entities.get(entity_name))
        source_records[entity_name] = records

    ## TODO: Leave the query by batch option commented out as it might be needed when we switch into requesting data from DISKOS (larger data)

In [7]:
"""2. Records ids uniqueness"""

entities_names = list(entities.keys())
entities_unique_identifiers = list([entity.get('enitity_unique_identifier') for entity in entities.values()])

ids_counts = dict()
for pair in list(zip(entities_names, entities_unique_identifiers)):
    ids_counts[pair[0]] = len(unique_ids_counts(source_records[pair[0]], pair[1]))
    
for key in ids_counts.keys():
    print(f"Entity: {key} has {len(source_records[key])} records, from which {ids_counts[key]} are have unique identifiers (ids)")

Entity: seismicsurveys has 1979 records, from which 1979 are have unique identifiers (ids)
Entity: seismicprojects has 2408 records, from which 2408 are have unique identifiers (ids)
Entity: navigationsets has 3257 records, from which 3257 are have unique identifiers (ids)
Entity: binsetgrids has 2148 records, from which 2148 are have unique identifiers (ids)
Entity: seismicpoststackdatasets has 19908 records, from which 19896 are have unique identifiers (ids)
Entity: poststackcubes has 19829 records, from which 19817 are have unique identifiers (ids)
Entity: poststackcubegeometries has 19829 records, from which 19817 are have unique identifiers (ids)


In [8]:
"""
2. Records ids uniqueness.
Let's find the non-unique ids for each entity.
"""

ids_duplicates = dict()
for pair in list(zip(entities_names, entities_unique_identifiers)):
    ids_duplicates[pair[0]] = get_non_unique_values(source_records[pair[0]], pair[1])
    
for key in ids_duplicates.keys():
    print(f"Entity: {key} - Duplicated ids: {len(ids_duplicates[key])}")

Entity: seismicsurveys - Duplicated ids: 0
Entity: seismicprojects - Duplicated ids: 0
Entity: navigationsets - Duplicated ids: 0
Entity: binsetgrids - Duplicated ids: 0
Entity: seismicpoststackdatasets - Duplicated ids: 12
Entity: poststackcubes - Duplicated ids: 12
Entity: poststackcubegeometries - Duplicated ids: 12


In [9]:
one_to_one, one_to_many, many_to_one, many_to_many, one_to_none, non_related = count_relationships(
    source_records["poststackcubegeometries"],
    source_records["seismicpoststackdatasets"],
    "seismicPoststackDatasetId"
)

print(f"One-to-One count: {len(one_to_one)}")
print(f"One-to-Many count: {len(one_to_many)}")
print(f"Many-to-One count: {len(many_to_one)}")
print(f"Many-to-Many count: {len(many_to_many)}")
print(f"One-to-None count: {len(one_to_none)}")

One-to-One count: 19805
One-to-Many count: 0
Many-to-One count: 0
Many-to-Many count: 12
One-to-None count: 0
